# Search for range of float model

### Import Packages

In [1]:
import tensorflow as tf
import json
from tabulate import tabulate
from tensorflow.keras import layers, models # type: ignore
from tensorflow.keras.applications import VGG16 # type: ignore
from tensorflow.keras.applications.vgg16 import preprocess_input # type: ignore

In [2]:
# Local Packages
from ml_project_util.path import path_definition
from ml_project_util.flatten_model import flatten_condtitional
from ml_project_util.quantization_util import activation_range_search, gen_sample_paths

### Variable Paths, Names, Execution Environments

In [3]:
model_name = 'CD4_P2_FT_003_val0.0336'

In [4]:
BASE_PATH, PATH_DATASET, PATH_RAWDATA, PATH_JOINEDDATA, PATH_SAVEDMODELS = path_definition()

### Load Float Model

In [5]:
short_name = model_name[:-10]
parent_name = model_name[:3]
filepath = f'{PATH_SAVEDMODELS}/{parent_name}/{model_name}.keras'
model = tf.keras.models.load_model(filepath)
model = flatten_condtitional(model, model_name)

### Activation Range for Random Input

In [13]:
import numpy as np
input_shape = (1, 224, 224, 3)  # batch size 1 here
input_data = np.random.rand(*input_shape).astype(np.float32)

x = tf.convert_to_tensor(input_data)

print("Activation ranges per layer:")

# Manually feed-forward through layers
for i, layer in enumerate(model.layers):
    x = layer(x)  # call layer on tensor input

    # x is a tensor, get numpy array for min/max
    activations = x.numpy()

    print(f"Layer {i} ({layer.name}): min={activations.min()}, max={activations.max()}")

Activation ranges per layer:
Layer 0 (block1_conv1): min=0.0, max=4.269227027893066
Layer 1 (block1_conv2): min=0.0, max=14.899660110473633
Layer 2 (block1_pool): min=0.0, max=14.899660110473633
Layer 3 (block2_conv1): min=0.0, max=32.683876037597656
Layer 4 (block2_conv2): min=0.0, max=69.7279281616211
Layer 5 (block2_pool): min=0.0, max=69.7279281616211
Layer 6 (block3_conv1): min=0.0, max=107.82502746582031
Layer 7 (block3_conv2): min=0.0, max=133.61187744140625
Layer 8 (block3_conv3): min=0.0, max=183.60520935058594
Layer 9 (block3_pool): min=0.0, max=183.60520935058594
Layer 10 (block4_conv1): min=0.0, max=94.03123474121094
Layer 11 (block4_conv2): min=0.0, max=55.30662155151367
Layer 12 (block4_conv3): min=0.0, max=30.583187103271484
Layer 13 (block4_pool): min=0.0, max=30.583187103271484
Layer 14 (block5_conv1): min=0.0, max=9.469626426696777
Layer 15 (block5_conv2): min=0.0, max=4.933740615844727
Layer 16 (block5_conv3): min=0.0, max=9.568266868591309
Layer 17 (block5_pool): mi

### Find Activation Range for Conv and Dense Layers & Store it

In [6]:
sampled_files = gen_sample_paths()
activation_sw_range_dict = activation_range_search(sampled_files, model, model_name, mode='s', force=0)

Read activation range json from C:/Users/arisi/Documents/VSCode/CatsDogs/Docs_Reports/Quant/Ranges/CD4_P2_FT_003_activation_sw_range.json
